In [3]:
import warnings
from typing import List

import numpy as np
import pandas as pd
import plotly.express as px
from autumn.core.inputs.database import get_input_db
from autumn.core.inputs.demography.queries import get_population_by_agegroup
from autumn.core.project import ParameterSet, get_project
from autumn.core.utils.display import pretty_print

In [4]:
warnings.filterwarnings("ignore")

## Modelled Population

In [5]:
input_db = get_input_db()
pop = input_db.query(table_name="population", conditions={"iso3": "KIR"})
get_population_by_agegroup([0, 5, 15, 35, 50], "KIR", year=1955)


def find_agebins(x, bins: List[int], labels: List[str]):
    return pd.cut(x, bins, labels=labels, right=False)


pop["age_bins"] = find_agebins(
    pop["start_age"], [0, 5, 15, 35, 50, 100], ["0-4", "5-14", "15-34", "35-49", "50+"]
)
pop = pop.groupby(["year", "age_bins"])["population"].agg(["sum"]).reset_index()

In [6]:
fig = px.area(
    pop,
    x="year",
    y="sum",
    color="age_bins",
    labels={"sum": "Population", "year": "Year", "age_bins": "Age Group"},
    title="Population of Kiribati over time",
)
fig.update_layout(title_x=0.5)
fig.show()

## Running an empty model

In [13]:
project = get_project("tb_dynamics", "kiribati")

In [14]:
base_params = project.param_set.baseline

In [16]:
model_0 = project.run_baseline_model(base_params)

In [17]:
df_0 = model_0.get_derived_outputs_df()

In [18]:
fig = px.line(
    df_0,
    x=df_0.index,
    y="population_size",
    labels={"population_size": "Population", "index": "Year"},
)
fig.show()